# XGBoost Regression Trials

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import warnings
from glob import glob
from time import time

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xgboost as xgb
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, plotting
from hyperopt.pyll.base import scope
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline

In [ ]:
%aimport src.ml_custom_transformers
import src.ml_custom_transformers as ct

%aimport src.xgboost_helpers
from src.xgboost_helpers import get_xgboost_training_curves

%aimport src.metrics_helpers
from src.metrics_helpers import rmspe_xg, rmspe, rmse

%aimport src.utils
from src.utils import get_xgboost_preds_obs

%aimport src.data_helpers
from src.data_helpers import sample_train_val_split, split_data

%aimport src.test_helpers
from src.test_helpers import (
    test_dfgetxy,
    test_split_data,
    test_fillna,
    test_sampled_data_sizes,
    test_target_encode_categorical_features,
    test_log1p,
)

%aimport src.hyperopt_helpers
from src.hyperopt_helpers import summarize_hyperopt_trials

%aimport src.visualization_helpers
from src.visualization_helpers import plot_qq

<a id="toc"></a>

## [Table of Contents](#table-of-contents)
0.  [About](#about)
1.  [User Inputs](#user-inputs)
2.  [Load Data](#load-data)
3.  [Preprocessing - Set index for Data](#preprocessing---set-index-for-data)
4.  [Create Training, Validation and Testing Splits from Training Data](#create-training-validation-and-testing-splits-from-training-data)
5.  [Show `datatype`s and Missing Values](#show-`datatype`s-and-missing-values)
    -   5.1. [Inspect Columns with Missing Values](#inspect-columns-with-missing-values)
6.  [Hyper-Parameter Tuning with K-Fold Cross-Validation](#hyper-parameter-tuning-with-k-fold-cross-validation)
    -   6.1. [Check Data Processing during Cross-Validation](#check-data-processing-during-cross-validation)
    -   6.2. [Tune HyperParameters](#tune-hyperparameters)
    -   6.3. [Get Best HyperParameters](#get-best-hyperparameters)
7. [Extract Features and Target from all Splits](#extract-features-and-target-from-all-splits)
8. [Handle missing values in each split - Use Median of Training Data](#handle-missing-values-in-each-split---use-median-of-training-data)
9.  [Encode Categorical Features](#encode-categorical-features)
10.  [Scaling Numerical Features](#scaling-numerical-features)
11.  [Transform Target](#transform-target)
12.  [Train-Predict with Best HyperParameters](#train-predict-with-best-hyperparameters)
     -   12.1. [Train with best hyper-parameters](#train-with-best-hyper-parameters)
     -   12.2. [XGBoost Training Curves - Training and Validation Splits](#xgboost-training-curves---training-and-validation-splits)
     -   12.3. [Make Predictions](#make-predictions)
13.  [Model Evaluation on Test Split](#model-evaluation-on-test-split)
     -   13.1. [Evaluation Metric](#evaluation-metric)
     -   13.2. [Distribution of Residuals](#distribution-of-residuals)
     -   13.3. [Prediction Error - Observed vs Predicted](#prediction-error---observed-vs-predicted)
     -   13.4. [Residuals vs Observed](#residuals-vs-observed)
     -   13.5. [Quantile-Quantile Plot for Residuals](#quantile-quantile-plot-for-residuals)
14.  [Export trained ML model](#export-trained-ml-model)
     -   16.1. [Train ML model with all available training data and no early-stopping](#train-ml-model-with-all-available-training-data-and-no-early-stopping)
     -   16.2. [Export ML model to file](#export-ml-model-to-file)
15. [Future Work](#future-work)

<a id="about"></a>

## 0. [About](#about)

In this notebook, we'll run ML regression trials using XGBoost.

<a id="user-inputs"></a>

## 1. [User Inputs](#user-inputs)

In [ ]:
PROJ_ROOT_DIR = os.getcwd()

In [ ]:
columns = [
    "Store",
    "DayOfWeek",
    "Year",
    "Month",
    "Day",
    "StateHoliday",
    "CompetitionMonthsOpen",
    "Promo2Weeks",
    "StoreType",
    "Assortment",
    "PromoInterval",
    "CompetitionOpenSinceYear",
    "Promo2SinceYear",
    "State",
    "Week",
    "Events",
    "Promo_fw",
    "Promo_bw",
    "StateHoliday_fw",
    "StateHoliday_bw",
    "SchoolHoliday_fw",
    "SchoolHoliday_bw",
    "CompetitionDistance",
    "Max_TemperatureC",
    "Mean_TemperatureC",
    "Min_TemperatureC",
    "Max_Humidity",
    "Mean_Humidity",
    "Min_Humidity",
    "Max_Wind_SpeedKm_h",
    "Mean_Wind_SpeedKm_h",
    "CloudCover",
    # "trend",
    # "trend_DE",
    "AfterStateHoliday",
    "BeforeStateHoliday",
    "Promo",
    "SchoolHoliday",
    "Date",
    "Sales",
]
continuous_vars_missing_vals = ["CompetitionDistance", "CloudCover"]

In [ ]:
processed_data_path = os.path.join(PROJ_ROOT_DIR, "data", "processed")
train_parquet_filepath = os.path.join(
    processed_data_path, "cleaned_train" + "_*" + ".parquet.gzip"
)
holdout_parquet_filepath = os.path.join(
    processed_data_path, "cleaned_test" + "_*" + ".parquet.gzip"
)
train_parquet_full_filepath = glob(train_parquet_filepath)[0]
print(train_parquet_full_filepath)
holdout_parquet_full_filepath = glob(holdout_parquet_filepath)[0]
print(holdout_parquet_full_filepath)

seed = 123
space = {
    # Learning rate: default 0.3 -> range: [0,3]
    "eta": hp.quniform("eta", 0.01, 0.3, 0.001),
    # Control complexity (control overfitting)
    # Maximum depth of a tree: default 6 -> range: [0:∞]
    "max_depth": scope.int(hp.quniform("max_depth", 5, 10, 1)),
    # Minimum sum of instance weight (hessian) needed in a child: default 1
    "min_child_weight": hp.quniform("min_child_weight", 1, 3, 1),
    # Minimum loss reduction required: default 0 -> range: [0,∞]
    "gamma": hp.quniform("gamma", 0, 5, 0.5),
    # Add randomness to make training robust to noise (control overfitting)
    # Subsample ratio of the training instance: default 1
    "subsample": hp.quniform("subsample", 0.5, 1, 0.05),
    # Subsample ratio of columns when constructing each tree: default 1
    "colsample_bytree": hp.quniform("colsample_bytree", 0.5, 1, 0.05),
    # Regression problem
    "objective": "reg:squarederror",
    # For reproducibility
    "seed": seed,
    # Faster computation = gpu_hist
    "tree_method": "hist",
}

In [ ]:
warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
def check_score():
    _ = score(space, True)

<a id="load-data"></a>

## 2. [Load Data](#load-data)

In [ ]:
%%time
# 1) Load data
tunning_dataset = pd.read_parquet(train_parquet_full_filepath, columns=columns)
holdout_dataset = pd.read_parquet(
    holdout_parquet_full_filepath, columns=list(set(columns) - (set(["Sales"])))
)
print(tunning_dataset.shape)
print(holdout_dataset.shape)

<a id="preprocessing---set-index-for-data"></a>

## 3. [Preprocessing - Set index for Data](#preprocessing---set-index-for-data)

In [ ]:
# %%time
# # 2) Let's use the date as the index and sort the data
# tunning_dataset.sort_values("Date", inplace=True)
# tunning_dataset.set_index("Date", inplace=True)
# columns.remove("Date")

# tunning_dataset_X = tunning_dataset
# tunning_dataset_y = tunning_dataset_X.pop("Sales")

# # 2) Let's use the date as the index and sort the data
# holdout_dataset.sort_values("Date", inplace=True)
# holdout_dataset.set_index("Date", inplace=True)

# holdout_dataset_X = holdout_dataset
# print(tunning_dataset.shape, tunning_dataset_X.shape, tunning_dataset_y.shape)

In [ ]:
%%time
setup_pipe = Pipeline(
    [
        ("datesort", ct.DFSortByDateSetDateIndex("Date")),
    ]
)
tunning_dataset = setup_pipe.fit_transform(tunning_dataset)
print(tunning_dataset.shape)
holdout_dataset = setup_pipe.fit_transform(holdout_dataset)
print(holdout_dataset.shape)

<a id="create-training-validation-and-testing-splits-from-training-data"></a>

## 4. [Create Training, Validation and Testing Splits from Training Data](#create-training-validation-and-testing-splits-from-training-data)

Although a holdout set is provided, the target value is not known there. So, we'll divide the overall training data into training, validation and testing splits, from the training data, in order to assist in model assessment.

Hyper-parameter tuning will be preformed using the training split, while the validation and testing split will only be used for assessment (with the best hyper-parameters).

In [ ]:
# n = len(holdout_dataset)
# idx = np.arange(0, len(tunning_dataset))
# idx.sort()
# train_idx_end = len(tunning_dataset) - (2 * n)
# val_idx_end = len(tunning_dataset) - (1 * n)
# train_index = idx[:train_idx_end]
# val_index = idx[train_idx_end:val_idx_end]
# test_index = idx[val_idx_end:]

In [ ]:
# %%time
# df_train, df_val, df_test = (
#     tunning_dataset.iloc[train_index].copy(),
#     tunning_dataset.iloc[val_index].copy(),
#     tunning_dataset.iloc[test_index].copy(),
# )
# print(len(df_train), len(df_val), len(df_test))
# assert len(df_val) == len(df_test)

In [ ]:
%%time
d = split_data(tunning_dataset, holdout_dataset)
train_index, val_index, test_index = d["indexes"]
df_train, df_val, df_test = d["splits"]
test_split_data(
    max(train_index),
    max(val_index),
    max(test_index),
    df_train,
    df_val,
    df_test,
    holdout_dataset,
)

<a id="show-`datatype`s-and-missing-values"></a>

## 5. [Show `datatype`s and Missing Values](#show-`datatype`s-and-missing-values)

In [ ]:
df_train_summary = (
    df_train.isna()
    .sum()
    .to_frame()
    .rename(columns={0: "NaNs"})
    .merge(
        df_train.dtypes.to_frame().rename(columns={0: "dtype"}),
        left_index=True,
        right_index=True,
    )
)
display(df_train.head())
display(df_train_summary)

<a id="inspect-columns-with-missing-values"></a>

### 5.1. [Inspect Columns with Missing Values](#inspect-columns-with-missing-values)

In [ ]:
df_train_with_nans = df_train[df_train.columns[(df_train.isna().sum() > 0).tolist()]]
display(df_train_with_nans.dtypes.to_frame().T)
display(df_train_with_nans.head())

In [ ]:
# # 3) Apply log transform
# tunning_dataset_y = np.log1p(tunning_dataset_y)

# # Number of cross-validation folds (from the last)
# pred_folds = 3
# train_times = []

<a id="hyper-parameter-tuning-with-k-fold-cross-validation"></a>

## 6. [Hyper-Parameter Tuning with K-Fold Cross-Validation](#hyper-parameter-tuning-with-k-fold-cross-validation)

In [ ]:
def score(params, check_data_run=False):
    n_split = 10
    train_sample_size = 5_000
    val_sample_size = 2_500
    rmspe_test_fold_scores = []
    tscv = TimeSeriesSplit(n_splits=n_split)
    for cv_fold_idx, (train_index, test_index) in enumerate(tscv.split(df_train)):
        assert cv_fold_idx + 1 <= n_split
        assert train_sample_size <= len(test_index)

        # 5) Select a random sample for early stopping
        (
            train_train_index,
            train_es_index,
            test_index_sampled,
            train_full_index,
        ) = sample_train_val_split(
            len(train_index), len(test_index), train_sample_size, val_sample_size
        )
        df_train_cv = df_train.iloc[train_train_index]
        df_val_cv = df_train.iloc[train_es_index]
        df_test_cv = df_train.iloc[test_index_sampled]
        if check_data_run:
            print(
                f"k={cv_fold_idx}, "
                f"Train={len(train_index)}, "
                f"Train_Sampled={len(df_train_cv)}, "
                f"Val={len(train_index)}, "
                f"Val_Sampled={len(df_train_cv)}, "
                f"Test={len(test_index)}, "
                f"TestSampled={len(df_test_cv)}"
            )
        # print(min(train_train_index), max(train_train_index))
        # print(min(train_es_index), max(train_es_index))
        # print(min(test_index_sampled), max(test_index_sampled))
        test_sampled_data_sizes(
            train_train_index,
            train_es_index,
            test_index_sampled,
            val_sample_size,
            train_full_index,
            df_train_cv,
            df_val_cv,
            df_test_cv,
        )

        # 4) Select data by index from the time series cross validatation split
        # X_train, X_val, X_test, y_train, y_val, y_test = (
        #     tunning_dataset_X.iloc[train_train_index].copy(),
        #     tunning_dataset_X.iloc[train_es_index].copy(),
        #     tunning_dataset_X.iloc[test_index_sampled].copy(),
        #     tunning_dataset_y.iloc[train_train_index].copy(),
        #     tunning_dataset_y.iloc[train_es_index].copy(),
        #     tunning_dataset_y.iloc[test_index_sampled].copy(),
        # )
        getxy_pipe = Pipeline(
            [
                ("getxy", ct.DFGetXY("Sales")),
            ]
        )
        X_train, y_train = getxy_pipe.fit_transform(df_train_cv.copy())
        X_val, y_val = getxy_pipe.fit_transform(df_val_cv.copy())
        X_test, y_test = getxy_pipe.fit_transform(df_test_cv.copy())
        test_dfgetxy(X_train, y_train, "Sales")
        test_dfgetxy(X_val, y_val, "Sales")
        test_dfgetxy(X_test, y_test, "Sales")

        # 5) Deal with missing continuous values
        # for col_name in continuous_vars_missing_vals:
        #     # Add na cols
        #     X_train[col_name + "_na"] = pd.isnull(X_train[col_name])
        #     X_val[col_name + "_na"] = pd.isnull(X_val[col_name])
        #     X_test[col_name + "_na"] = pd.isnull(X_test[col_name])
        #     # Fill missing with median (default in FastAI)
        #     fillter = X_train[col_name].median()
        #     X_train[col_name] = X_train[col_name].fillna(fillter)
        #     X_val[col_name] = X_val[col_name].fillna(fillter)
        #     X_test[col_name] = X_test[col_name].fillna(fillter)
        fillna_pipe = Pipeline(
            [
                ("cd", ct.DFFillNa("CompetitionDistance")),
                ("cda", ct.DFAddNaIndicator("CompetitionDistance")),
                ("cc", ct.DFFillNa("CloudCover")),
                ("cca", ct.DFAddNaIndicator("CloudCover")),
            ]
        )
        fillna_pipe.fit(X_train)
        X_train = fillna_pipe.transform(X_train)
        X_val = fillna_pipe.transform(X_val)
        X_test = fillna_pipe.transform(X_test)
        # assert (X_train[continuous_vars_missing_vals].isna().sum().tolist()) == [
        #     0
        # ] * len(continuous_vars_missing_vals)
        test_fillna(X_train, X_val, X_test, continuous_vars_missing_vals)

        # 6) Deal with categorical variables
        # cat_vars = list(X_train.select_dtypes(include="object"))
        # te = TargetEncoder(handle_missing="value")
        # X_train = te.fit_transform(X_train, cols=cat_vars, y=y_train)
        # X_val = te.transform(X_val)
        # X_test = te.transform(X_test)
        cat_vars = list(X_train.select_dtypes(include="object"))
        cat_pipe = Pipeline(
            [
                ("targetcats", ct.DFTargetEncodeCategoricalFeatures(cat_vars)),
            ]
        )
        cat_pipe.fit(X_train, y_train)
        X_train = cat_pipe.transform(X_train)
        X_val = cat_pipe.transform(X_val)
        X_test = cat_pipe.transform(X_test)
        # assert not list(X_train.select_dtypes(include="object"))
        # assert not list(X_val.select_dtypes(include="object"))
        # assert not list(X_test.select_dtypes(include="object"))
        test_target_encode_categorical_features(X_train, X_val, X_test)

        # 3) Apply log transform
        # y_train = np.log1p(y_train)
        # y_val = np.log1p(y_val)
        # y_test = np.log1p(y_test)
        log1p_pipe = Pipeline(
            [
                ("log1p", ct.DFLog1p("Sales")),
            ]
        )
        y_train = log1p_pipe.fit_transform(y_train.to_frame()).squeeze()
        y_val = log1p_pipe.fit_transform(y_val.to_frame()).squeeze()
        y_test = log1p_pipe.fit_transform(y_test.to_frame()).squeeze()
        test_log1p([y_train, y_val, y_test])

        # 7) Convert to DMatrix for XGBoost
        dtrain = xgb.DMatrix(X_train, y_train)
        dvalid = xgb.DMatrix(X_val, y_val)
        dtest = xgb.DMatrix(X_test)

        watchlist = [(dtrain, "train"), (dvalid, "eval")]

        if not check_data_run:
            # Can use feval for a custom objective function
            model = xgb.train(
                params,
                dtrain,
                early_stopping_rounds=50,
                num_boost_round=400,
                verbose_eval=False,
                feval=rmspe_xg,
                evals=watchlist,
            )
            # validation - this will be the score that we append to a list,
            # which will be fed as the score? Is all of this the score?
            y_pred = model.predict(dtest)
            rmspe_test = rmspe(y_test, y_pred)

            rmspe_test_fold_scores.append(rmspe_test)

    if not check_data_run:
        return np.mean(rmspe_test_fold_scores)
    return [X_train, X_val, X_test, y_train, y_val, y_test]

<a id="check-data-processing-during-cross-validation"></a>

### 6.1. [Check Data Processing during Cross-Validation](#check-data-processing-during-cross-validation)

In [ ]:
%%time
check_score()

<a id="tune-hyperparameters"></a>

### 6.2. [Tune HyperParameters](#tune-hyperparameters)

In [ ]:
%%time
# trials will contain logging information
trials = Trials()

# tune hyper-parameters
best_hyperparams = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=100)
print(f"Best hyper-parameters: {best_hyperparams}")
display(summarize_hyperopt_trials(trials))

<a id="get-best-hyperparameters"></a>

### 6.3. [Get Best HyperParameters](#get-best-hyperparameters)

In [ ]:
# best_hyperparams = {}
best_hyperparams["objective"] = "reg:squarederror"
best_hyperparams["seed"] = seed
best_hyperparams["tree_method"] = "hist"
best_hyperparams["max_depth"] = int(best_hyperparams["max_depth"])
# best_hyperparams["max_depth"] = 8

<a id="extract-features-and-target-from-all-splits"></a>

## 7. [Extract Features and Target from all Splits](#extract-features-and-target-from-all-splits)

In [ ]:
# %%time
# X_train, X_val, X_test = (
#     tunning_dataset_X.iloc[train_index].copy(),
#     tunning_dataset_X.iloc[val_index].copy(),
#     tunning_dataset_X.iloc[test_index].copy(),
# )
# y_train, y_val, y_test = (
#     tunning_dataset_y.iloc[train_index].copy(),
#     tunning_dataset_y.iloc[val_index].copy(),
#     tunning_dataset_y.iloc[test_index].copy(),
# )
# print(
#     X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape
# )

In [ ]:
%%time
getxy_pipe = Pipeline(
    [
        ("getxy", ct.DFGetXY("Sales")),
    ]
)
X_train, y_train = getxy_pipe.fit_transform(df_train.copy())
X_val, y_val = getxy_pipe.fit_transform(df_val.copy())
X_test, y_test = getxy_pipe.fit_transform(df_test.copy())
print(
    X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape
)

<a id="handle-missing-values-in-each-split---use-median-of-training-data"></a>

## 8. [Handle missing values in each split - Use Median of Training Data](#handle-missing-values-in-each-split---use-median-of-training-data)

In [ ]:
# %%time
# for col_name in ["CompetitionDistance", "CloudCover"]:
#     # Add na cols
#     X_train[col_name + "_na"] = pd.isnull(X_train[col_name])
#     X_val[col_name + "_na"] = pd.isnull(X_val[col_name])
#     X_test[col_name + "_na"] = pd.isnull(X_test[col_name])
#     # Fill missing with median (default in FastAI) of TRAINING set
#     fillter = X_train[col_name].median()
#     X_train[col_name] = X_train[col_name].fillna(fillter)
#     X_val[col_name] = X_val[col_name].fillna(fillter)
#     X_test[col_name] = X_test[col_name].fillna(fillter)
# assert (X_train[continuous_vars_missing_vals].isna().sum().tolist()) == [0] * len(
#     continuous_vars_missing_vals
# )

In [ ]:
%%time
fillna_pipe = Pipeline(
    [
        ("cd", ct.DFFillNa("CompetitionDistance")),
        ("cda", ct.DFAddNaIndicator("CompetitionDistance")),
        ("cc", ct.DFFillNa("CloudCover")),
        ("cca", ct.DFAddNaIndicator("CloudCover")),
    ]
)
fillna_pipe.fit(X_train)
X_train = fillna_pipe.transform(X_train)
X_val = fillna_pipe.transform(X_val)
X_test = fillna_pipe.transform(X_test)
test_fillna(X_train, X_val, X_test, continuous_vars_missing_vals)

<a id="encode-categorical-features"></a>

## 9. [Encode Categorical Features](#encode-categorical-features)

In [ ]:
# %%time
# cat_vars = list(X_train.select_dtypes(include="object"))
# te = TargetEncoder(handle_missing="value")
# X_train = te.fit_transform(X_train, cols=cat_vars, y=y_train)
# X_val = te.transform(X_val)
# X_test = te.transform(X_test)
# assert not list(X_train.select_dtypes(include="object"))
# assert not list(X_val.select_dtypes(include="object"))
# assert not list(X_test.select_dtypes(include="object"))

In [ ]:
%%time
cat_vars = list(X_train.select_dtypes(include="object"))
cat_pipe = Pipeline(
    [
        ("targetcats", ct.DFTargetEncodeCategoricalFeatures(cat_vars)),
    ]
)
cat_pipe.fit(X_train, y_train)
X_train = cat_pipe.transform(X_train)
X_val = cat_pipe.transform(X_val)
X_test = cat_pipe.transform(X_test)
test_target_encode_categorical_features(X_train, X_val, X_test)

<a id="scaling-numerical-features"></a>

## 10. [Scaling Numerical Features](#scaling-numerical-features)

<span style='color:red'><b>(IMPORTANT) To be done</b></span>

<a id="transform-target"></a>

## 11. [Transform Target](#transform-target)

In [ ]:
# y_train = np.log1p(y_train)
# y_val = np.log1p(y_val)
# y_test = np.log1p(y_test)

In [ ]:
%%time
log1p_pipe = Pipeline(
    [
        ("log1p", ct.DFLog1p("Sales")),
    ]
)
y_train = log1p_pipe.fit_transform(y_train.to_frame()).squeeze()
y_val = log1p_pipe.fit_transform(y_val.to_frame()).squeeze()
y_test = log1p_pipe.fit_transform(y_test.to_frame()).squeeze()
test_log1p([y_train, y_val, y_test])

<a id="train-predict-with-best-hyperparameters"></a>

## 12. [Train-Predict with Best HyperParameters](#train-predict-with-best-hyperparameters)

Convert overall data splits to XGBoost `DMatrix` objects

In [ ]:
%%time
dtrain = xgb.DMatrix(X_train, y_train)
dvalid = xgb.DMatrix(X_val, y_val)
dtest = xgb.DMatrix(X_test)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]

<a id="train-with-best-hyper-parameters"></a>

### 12.1. [Train with best hyper-parameters](#train-with-best-hyper-parameters)

In [ ]:
%%time
training_curves_002 = {}
model_exp002 = xgb.train(
    params=best_hyperparams, 
    dtrain=dtrain, 
    num_boost_round=400, 
    early_stopping_rounds=50, 
    feval=rmspe_xg, 
    evals=watchlist, 
    evals_result=training_curves_002
)

<a id="xgboost-training-curves---training-and-validation-splits"></a>

### 12.2. [XGBoost Training Curves - Training and Validation Splits](#xgboost-training-curves---training-and-validation-splits)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4))
df_training_curves = get_xgboost_training_curves(training_curves_002, "rmspe")
df_training_curves.plot(ax=ax)
ax.grid()
ax.legend(ncol=2, bbox_to_anchor=(0.8, 1.1), loc="upper left", frameon=False)
ax.set_title("RMSPE Loss vs Iterations", loc="left", fontweight="bold")

<a id="make-predictions"></a>

### 12.3. [Make Predictions](#make-predictions)

In [ ]:
%%time
predictions_002 = model_exp002.predict(dtest)
predictions_002_train = model_exp002.predict(dtrain)

<a id="model-evaluation-on-test-split"></a>

## 13. [Model Evaluation on Test Split](#model-evaluation-on-test-split)

In [ ]:
%%time
df_pred = get_xgboost_preds_obs(y_test, predictions_002).set_index("Date_x")
df_pred["Sales_Raw"] = np.expm1(df_pred["Sales"])
df_pred["pred_Raw"] = np.expm1(df_pred["pred"])
df_pred["res"] = df_pred["pred_Raw"] - df_pred["Sales_Raw"]

df_pred_train = get_xgboost_preds_obs(y_train, predictions_002_train).set_index(
    "Date_x"
)
df_pred_train["Sales_Raw"] = np.expm1(df_pred_train["Sales"])
df_pred_train["pred_Raw"] = np.expm1(df_pred_train["pred"])
df_pred_train["res"] = df_pred_train["pred_Raw"] - df_pred_train["Sales_Raw"]

<a id="evaluation-metric"></a>

### 13.1. [Evaluation Metric](#evaluation-metric)

In [ ]:
%%time
metric_test = rmspe(df_pred["pred"], df_pred["Sales"])
metric_train = rmspe(df_pred_train["pred"], df_pred_train["Sales"])
rmse_test = rmse(df_pred["pred"], df_pred["Sales"])
rmse_train = rmse(df_pred_train["pred"], df_pred_train["Sales"])
d_metrics = {
    "rmse_train": rmse_train,
    "rmse_test": rmse_test,
    "rmspe_train": metric_train,
    "rmspe_test": metric_test,
}
df_metrics = pd.DataFrame.from_dict(d_metrics, orient="index").T
display(df_metrics)

### Observed and Predicted, as a function of time

In [ ]:
%%time
fig, ax = plt.subplots(1, 1, figsize=(12, 4))
np.expm1(df_pred[["Sales", "pred"]]).head(25_000).plot(ax=ax)
ax.grid()
ax.set_xlabel(None)
ax.legend(ncol=2, bbox_to_anchor=(0.8, 1.1125), loc="upper left", frameon=False)
ax.set_title("Obs vs Pred", loc="left", fontweight="bold")

<a id="distribution-of-residuals"></a>

### 13.2. [Distribution of Residuals](#distribution-of-residuals)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
df_pred["res"].plot.hist(edgecolor="white", color="blue", ax=ax)
ax.set_ylabel(None)
ax.grid(color="lightgrey", alpha=1)
ax.set_title("Distribution of Residuals", fontweight="bold", loc="left")

<a id="prediction-error---observed-vs-predicted"></a>

### 13.3. [Prediction Error - Observed vs Predicted](#prediction-error---observed-vs-predicted)

In [ ]:
reg = LinearRegression()
reg.fit(df_pred[["pred_Raw"]], df_pred["Sales_Raw"])
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
df_pred[["Sales_Raw", "pred_Raw"]].plot.scatter(
    x="pred_Raw", y="Sales_Raw", color="white", edgecolors="b", s=40, ax=ax
)
ax.plot(
    df_pred["pred_Raw"].to_numpy(),
    reg.predict(df_pred[["pred_Raw"]]),
    label=f"best fit (R2={reg.score(df_pred[['pred_Raw']], df_pred['Sales_Raw']):.2f})",
    color="darkred",
    linestyle="--",
)
ax.set_xlabel(None)
ax.set_ylabel(None)
ax.grid(color="lightgrey", alpha=0.75)
ax.set_title(
    f"Observed vs Predicted (RMSPE={metric_test:.2f})", fontweight="bold", loc="left"
)
ax.axline([0, 0], [1, 1], label="identity", color="darkorange", linestyle="--")
ax.legend(frameon=False)
ax.set_xlim((0, None))
ax.set_ylim((0, None))

<a id="residuals-vs-observed"></a>

### 13.4. [Residuals vs Observed](#residuals-vs-observed)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
df_pred.loc[df_pred["res"] > -10_000][["Sales_Raw", "res"]].plot.scatter(
    x="Sales_Raw", y="res", color="white", edgecolors="b", s=40, ax=ax
)
ax.set_xlabel(None)
ax.set_ylabel(None)
ax.axline([0, 0], [1, 0], color="black", linestyle="--")
ax.grid(color="lightgrey", alpha=0.75)
ax.set_title("Residual vs Observed", fontweight="bold", loc="left")

<a id="quantile-quantile-plot-for-residuals"></a>

### 13.5. [Quantile-Quantile Plot for Residuals](#quantile-quantile-plot-for-residuals)

In [ ]:
plot_qq(
    df_pred["res"],
    fig_size=(6, 6),
)

<a id="export-trained-ml-model"></a>

## 14. [Export trained ML model](#export-trained-ml-model)

<a id="train-ml-model-with-all-available-training-data-and-no-early-stopping"></a>

### 14.1. [Train ML model with all available training data and no early-stopping](#train-ml-model-with-all-available-training-data-and-no-early-stopping)

Next, get the actual number of rounds where early stopping was triggered and use this for training the model on all the available training data - at this time, early stopping rounds will no longer be required since we know the exact round (`num_boost_round_best` below) when the reduction in loss has stopped.

In [ ]:
num_boost_round_best = model_exp002.best_iteration + 1
print(num_boost_round_best)

Next, train the model on all the available training data. This means re-running all the processing steps in this notebook (listed below, excluding those that have a line drawn through them since no testing data willl be available and no early stopping is required), using the full training dataset
- Extract Features and Target from full training data
- Handle missing values in full training data - Use Median of full training data
- Encode Categorical Features
- Scaling Numerical Features
- Transform Target
- Train with Best HyperParameters and Export to disk in preparation for inference
  - Train with best hyper-parameters
  - ~~XGBoost Training Curves~~
  - ~~Make predictions~~
  - Export trained ML model to file

<span style='color:red'><b>To be done</b></span>

<a id="export-ml-model-to-file"></a>

### 14.2. [Export ML model to file](#export-ml-model-to-file)

Finally, [export the model to a `pickle` file](https://cloud.google.com/ai-platform/prediction/docs/getting-started-scikit-xgboost#xgboost)

In [ ]:
import pickle

with open("model.pkl", "wb") as model_file:
    pickle.dump(model_exp002, model_file)

<a id="future-work"></a>

## 15. [Future Work](#future-work)

1.  <span style='color:red'><b>(IMPORTANT) Scale features (standardization or normalization)</b></span> during the following steps
    -   [final model training](#scaling-numerical-features)
    -   [cross-validation](#hyper-parameter-tuning-with-k-fold-cross-validation)
2.  Try making the regression target (`y`) stationary (in a timeseries sense)
3.  Examine prediction errors (eg. over- or under-predictions), grouped by
    -   store
    -   state
    -   school holiday
    -   state holiday
    -   other categorical features
4.  Explore XGBoost Hyper-Parameter settings during [cross-validation](#hyper-parameter-tuning-with-k-fold-cross-validation)
    -   increase number of estimators (`num_boosting_rounds`)
    -   modify `early_stopping_rounds`
    -   other XGBoost hyper-parameters (`eta`, `max_depth`)
5.  Although use-case calls for single-shot forecast, also consider a rolling forecast using ML algorithms (such as XGBoost, etc.) using one of the following two approaches
    -   recursive
        -   predict one-step ahead and add predicted value to training data for predicting next step
        -   disadvantage
            -   quickly accumulates prediction errors at each step since previously predicted values are used to build features for current step prediction
            -   not the best for long forecast horizons where accumulated error may render forecast unusable
    -   direct
        -   shift timeseries ahead by required horizon into future and then train model and make multi-step predictions for each step comprising the forecast horizon
        -   disadvantage
            -   can't model relationship between predictions since model for each step is independent
            -   larger computational cost so it is not suitable for long forecast horizons
            -   introduces a gap to be left **after** training data, of the same length as the forecast horizon, before first forecasted value is available
                -   this means that the training data set would need to end on June 14, 2015, which is six weeks (length of forecast horizon) before current end date (July 25, 2015) of training data and the first forecasted date will be July 25, 2015

    In both of these rolling forecast approaches, the forecast will need to also cover the gap (July 26, 2015 - July 31, 2015) period between the end of the training and start of the unknown (real) data.

---

<span style="float:left;">
    <a href="./1_data_clean_feat_eng.ipynb"><< 1 - Data Cleaning and Feature Engineering</a>
</span>

<span style="float:right;">
    &#169; 2021 | <a href="https://github.com/edesz/streetcar-delays">@edesz</a> (MIT)
</span>